# COVID-19 Classification

In this notebook, we have implemented baseline models to compare the accuracy of our proposed approach

1. Logistic Regression
2. Transfer Learning
3. Convolutional Neural Networks

we have also taken their last layer embeddings to analyse the decision boundaries using clustering approach

In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import numpy as np
import os
import shutil
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import f1_score

In [3]:
! pip install opencv-python

In [4]:
import os
import cv2
base_dir = './Covid19-dataset/'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test')

# Directory with our training cat/dog pictures
train_covid_dir = os.path.join(train_dir, 'Covid')
train_normal_dir = os.path.join(train_dir, 'Normal')
train_pneumonia_dir = os.path.join(train_dir, 'Viral Pneumonia')

# Directory with our validation cat/dog pictures
validation_covid_dir = os.path.join(validation_dir, 'Covid')
validation_normal_dir = os.path.join(validation_dir, 'Normal')
validation_pneumonia_dir = os.path.join(validation_dir, 'Viral Pneumonia')

In [5]:
train_covid_fnames = os.listdir(train_covid_dir)
train_normal_fnames = os.listdir(train_normal_dir)
train_pneumonia_fnames = os.listdir(train_pneumonia_dir)

print (len(train_covid_fnames)+len(train_normal_fnames)+len(train_pneumonia_fnames))
print (len(os.listdir(validation_covid_dir))+len(os.listdir(validation_normal_dir))+len(os.listdir(validation_pneumonia_dir)))

251
66


In [6]:
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# Parameters for our graph; we'll output images in a 4x4 configuration

nrows = 6
ncols = 4

pic_index = 0 # Index for iterating over images

In [7]:
from glob import glob

BASE_DATA_FOLDER = "./Covid19-dataset"
TRAin_DATA_FOLDER = os.path.join(BASE_DATA_FOLDER, "train")

images = []
labels = []

for class_folder_name in os.listdir(TRAin_DATA_FOLDER):
    class_folder_path = os.path.join(TRAin_DATA_FOLDER, class_folder_name)
    for image_path in glob(os.path.join(class_folder_path, "*.jpeg")):
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        
        #image = cv2.resize(image, (150, 150))
        #image = segment_plant(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (224,224))
        image = np.stack((image,)*3, axis=-1)
#         image = image.flatten()
        
        images.append(image)
        labels.append(class_folder_name)
    for image_path in glob(os.path.join(class_folder_path, "*.png")):
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        
        #image = cv2.resize(image, (150, 150))
        #image = segment_plant(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (224,224))
        image = np.stack((image,)*3, axis=-1)
        
#         image = image.flatten()
        
        images.append(image)
        labels.append(class_folder_name)
    for image_path in glob(os.path.join(class_folder_path, "*.jpg")):
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        
        #image = cv2.resize(image, (150, 150))
        #image = segment_plant(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (224,224))
        image = np.stack((image,)*3, axis=-1)
        
#         image = image.flatten()
        
        images.append(image)
        labels.append(class_folder_name)
        
images = np.array(images)
labels = np.array(labels)


# print (labels)
from sklearn import preprocessing
pre = preprocessing.LabelEncoder()
pre.fit(labels)
labels_numeric = pre.transform(labels)
Num_Class = 3

def OneHotEncoded(y_train):
    y_t=np.zeros((len(y_train),Num_Class), dtype=int)
    for i,x in enumerate(y_train):
        y_t[i][int(x)-1]=1
    return y_t
# print (labels)

labels = OneHotEncoded(labels_numeric)

X_train, X_test= train_test_split(images, test_size=0.2, random_state=42)
y_train, y_test= train_test_split(labels, test_size=0.2, random_state=42)

# print (labels_numeric)

print (X_train.shape)
print (X_test.shape)
print (y_train.shape)
print (y_test.shape)
X_train = X_train.reshape(200, 224, 224, 3)
X_test = X_test.reshape(51, 224, 224, 3)




(200, 224, 224, 3)
(51, 224, 224, 3)
(200, 3)
(51, 3)


## the next step is to define the model that will be trained to recognize covid, normal or Viral Pneumonia from these images

In [8]:
train_vgg_bf = X_train.reshape(X_train.shape[0],-1)
valid_vgg_bf = X_test.reshape(X_test.shape[0],-1)
print (valid_vgg_bf.shape)
print (train_vgg_bf.shape)

(51, 150528)
(200, 150528)


In [9]:
compare_loss={}
compare_accuracy = {}
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=147)
logreg.fit(train_vgg_bf, (y_train * range(Num_Class)).sum(axis=1))
valid_probs = logreg.predict_proba(valid_vgg_bf)
valid_preds = logreg.predict(valid_vgg_bf)
compare_loss['Vgg16']=log_loss(y_test, valid_probs)
compare_accuracy['Vgg16']=accuracy_score((y_test * range(Num_Class)).sum(axis=1), valid_preds)
print('Validation VGG LogLoss {}'.format(compare_loss['Vgg16']))
print('Validation VGG Accuracy {}'.format(compare_accuracy['Vgg16']))
y_lr_test = (y_test * range(Num_Class)).sum(axis=1)

Validation VGG LogLoss 1.1669545136449881
Validation VGG Accuracy 0.8823529411764706


In [10]:

print ('Accuracy:', accuracy_score(y_lr_test, valid_preds))
print ('F1 score:', f1_score(y_lr_test, valid_preds, average='macro'))
print ('Recall:', recall_score(y_lr_test, valid_preds, average='macro'))
print ('Precision:', precision_score(y_lr_test, valid_preds, average='macro'))
print ('\n clasification report:\n', classification_report(y_lr_test,valid_preds))
print ('\n confussion matrix:\n',confusion_matrix(y_lr_test, valid_preds))

X = train_vgg_bf
numpy_labels = y_train
numpy_all = logreg.predict_proba(X)

Accuracy: 0.8823529411764706
F1 score: 0.8833402260821615
Recall: 0.8838665290677675
Precision: 0.8911764705882352

 clasification report:
               precision    recall  f1-score   support

           0       0.82      0.93      0.87        15
           1       1.00      0.82      0.90        17
           2       0.85      0.89      0.87        19

    accuracy                           0.88        51
   macro avg       0.89      0.88      0.88        51
weighted avg       0.89      0.88      0.88        51


 confussion matrix:
 [[14  0  1]
 [ 1 14  2]
 [ 2  0 17]]


In [11]:
clusterer = KMeans(n_clusters=3)
preds = clusterer.fit_predict(X)
centers = clusterer.cluster_centers_
score = silhouette_score(X, preds)
print ("Sillouette Score using K-Means", score)


# from sklearn.mixture import GaussianMixture
# gm = GaussianMixture(n_components=3, random_state=0).fit(X)
# preds = gm.predict(X)
# score = silhouette_score(X, preds)
# print ("Sillouette Score using GMM", score)


Sillouette Score using K-Means 0.1567434865534098


IndexError: too many indices for array

In [12]:
from keras.applications.vgg16 import VGG16
from keras.applications import xception
from keras.applications import inception_v3
POOLING = 'avg'
vgg_bottleneck = VGG16(weights='imagenet', include_top=False, pooling=POOLING)
train_vgg_bf = vgg_bottleneck.predict(X_train, batch_size=32, verbose=1)
valid_vgg_bf = vgg_bottleneck.predict(X_test, batch_size=32, verbose=1)
print('VGG train bottleneck features shape: {} size: {:,}'.format(train_vgg_bf.shape, train_vgg_bf.size))
print('VGG valid bottleneck features shape: {} size: {:,}'.format(valid_vgg_bf.shape, valid_vgg_bf.size))


51/51 [==============================] - 5s 90ms/step
VGG train bottleneck features shape: (200, 512) size: 102,400
VGG valid bottleneck features shape: (51, 512) size: 26,112


In [13]:
compare_loss={}
compare_accuracy = {}
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=147)
logreg.fit(train_vgg_bf, (y_train * range(Num_Class)).sum(axis=1))
valid_probs = logreg.predict_proba(valid_vgg_bf)
valid_preds = logreg.predict(valid_vgg_bf)
compare_loss['Vgg16']=log_loss(y_test, valid_probs)
compare_accuracy['Vgg16']=accuracy_score((y_test * range(Num_Class)).sum(axis=1), valid_preds)
print('Validation VGG LogLoss {}'.format(compare_loss['Vgg16']))
print('Validation VGG Accuracy {}'.format(compare_accuracy['Vgg16']))
y_tl_test = (y_test * range(Num_Class)).sum(axis=1)

Validation VGG LogLoss 0.44055241175848586
Validation VGG Accuracy 0.9215686274509803


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [14]:
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import f1_score
print ('Accuracy:', accuracy_score(y_tl_test, valid_preds))
print ('F1 score:', f1_score(y_tl_test, valid_preds, average='macro'))
print ('Recall:', recall_score(y_tl_test, valid_preds, average='macro'))
print ('Precision:', precision_score(y_tl_test, valid_preds, average='macro'))
print ('\n clasification report:\n', classification_report(y_tl_test,valid_preds))
print ('\n confussion matrix:\n',confusion_matrix(y_tl_test, valid_preds))

Accuracy: 0.9215686274509803
F1 score: 0.9180555555555556
Recall: 0.9215686274509803
Precision: 0.9290382819794584

 clasification report:
               precision    recall  f1-score   support

           0       0.88      1.00      0.94        15
           1       1.00      0.76      0.87        17
           2       0.90      1.00      0.95        19

    accuracy                           0.92        51
   macro avg       0.93      0.92      0.92        51
weighted avg       0.93      0.92      0.92        51


 confussion matrix:
 [[15  0  0]
 [ 2 13  2]
 [ 0  0 19]]


In [15]:
import tensorflow as tf
 
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dense(3, activation='softmax')  
])


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [17]:
model.compile(optimizer='adam', loss= 'categorical_crossentropy', metrics=['accuracy'])

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode  = 'categorical',
                                                         target_size = (150, 150),
                                                         shuffle=False)

Found 251 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [19]:
history = model.fit(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=10,
                              epochs=20,
                              validation_steps = 3,
                              verbose=2)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 10 steps, validate for 3 steps
Epoch 1/20
10/10 - 9s - loss: 1.4507 - accuracy: 0.3246 - val_loss: 0.9754 - val_accuracy: 0.7500
Epoch 2/20
10/10 - 9s - loss: 0.8378 - accuracy: 0.7225 - val_loss: 0.6352 - val_accuracy: 0.6667
Epoch 3/20
10/10 - 9s - loss: 0.3404 - accuracy: 0.8691 - val_loss: 0.5681 - val_accuracy: 0.6667
Epoch 4/20
10/10 - 9s - loss: 0.2473 - accuracy: 0.9110 - val_loss: 0.5623 - val_accuracy: 0.8333
Epoch 5/20
10/10 - 9s - loss: 0.2324 - accuracy: 0.9110 - val_loss: 0.6192 - val_accuracy: 0.7667
Epoch 6/20
10/10 - 9s - loss: 0.2279 - accuracy: 0.9300 - val_loss: 0.7814 - val_accuracy: 0.7167
Epoch 7/20
10/10 - 9s - loss: 0.1542 - accuracy: 0.9424 - val_loss: 0.2892 - val_accuracy: 0.8667
Epoch 8/20
10/10 - 8s - loss: 0.1170 - accuracy: 0.9550 - val_loss: 0.4771 - val_accuracy: 0.8167
Epoch 9/20
10/10 - 9s - loss: 0.0784 - accuracy: 0.9738 - val_loss: 0.2287 - val_accuracy: 0.9000
Epoch 10/20
10/10 - 8s - lo

In [160]:
from sklearn.metrics import classification_report

y_pred = model.predict(validation_generator, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
# print (y_test)
from sklearn import preprocessing
pre = preprocessing.LabelEncoder()
pre.fit(y_test)
labels = pre.transform(y_test)
print (validation_generator.classes)
print (y_pred_bool)
# 
print(classification_report(validation_generator.classes, y_pred_bool))

numpy_labels = labels
numpy_all = logreg.coef_
print (numpy_all)
clusterer = KMeans(n_clusters=3)
preds = clusterer.fit_predict(X)
centers = clusterer.cluster_centers_
score = silhouette_score(X, preds)
print ("Sillouette Score using CNN approach", score)
# plot_mnist(numpy_all, numpy_labels,name="./embeddings_plot.png")

4/4 [==============================] - 2s 426ms/step
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 2 1 1 1 2 1
 1 1 0 1 1 1 1 1 1 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 1]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98        26
           1       0.89      0.85      0.87        20
           2       0.90      0.90      0.90        20

    accuracy                           0.92        66
   macro avg       0.92      0.92      0.92        66
weighted avg       0.92      0.92      0.92        66



In [20]:
y_test = (y_test * range(3)).sum(axis=1)
print ('Accuracy:', accuracy_score(y_test, valid_preds))
print ('F1 score:', f1_score(y_test, valid_preds, average='macro'))
print ('Recall:', recall_score(y_test, valid_preds, average='macro'))
print ('Precision:', precision_score(y_test, valid_preds, average='macro'))
print ('\n clasification report:\n', classification_report(y_test,valid_preds))
print ('\n confussion matrix:\n',confusion_matrix(y_test, valid_preds))

Accuracy: 0.9215686274509803
F1 score: 0.9180555555555556
Recall: 0.9215686274509803
Precision: 0.9290382819794584

 clasification report:
               precision    recall  f1-score   support

           0       0.88      1.00      0.94        15
           1       1.00      0.76      0.87        17
           2       0.90      1.00      0.95        19

    accuracy                           0.92        51
   macro avg       0.93      0.92      0.92        51
weighted avg       0.93      0.92      0.92        51


 confussion matrix:
 [[15  0  0]
 [ 2 13  2]
 [ 0  0 19]]
